In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Data formats
"ratings.dat" file format - UserID::MovieID::Rating::Timestamp
- UserIDs range between 1 and 6040 
- MovieIDs range between 1 and 3952
- Ratings are made on a 5-star scale (whole-star ratings only)
- Timestamp is represented in seconds since the epoch as returned by time(2)
- Each user has at least 20 ratings


"movies.dat" format - MovieID::Title::Genres
- Genres are pipe-separated and are selected from the following genres: [Action, Adventure, Animation, Children's, Comedy, Crime, Documentary, Drama, Fantasy, Film-Noir, Horror, Musical, Mystery, Romance, Sci-Fi, Thriller, War, Western]
- Titles are identical to titles provided by the IMDB (including
year of release)
- Some MovieIDs do not correspond to a movie due to accidental duplicate
entries and/or test entries
- Movies are mostly entered by hand, so errors and inconsistencies may exist


"users.dat" file format - UserID::Gender::Age::Occupation::Zip-code
- Gender is denoted by a "M" for male and "F" for female
- Age is chosen from the following ranges: {1:  "Under 18", 18:  "18-24", 25:  "25-34", 35:  "35-44", 45:  "45-49", 50:  "50-55", 56:  "56+"}

- Occupation is chosen from the following choices: {0:  "other" or "not specified", 1:  "academic/educator", 2:  "artist", 3:  "clerical/admin", 4:  "college/grad student", 5:  "customer service", 6:  "doctor/health care", 7:  "executive/managerial", 8:  "farmer", 9:  "homemaker", 10:  "K-12 student", 11:  "lawyer", 12:  "programmer", 13:  "retired", 14:  "sales/marketing", 15:  "scientist", 16:  "self-employed", 17:  "technician/engineer", 18:  "tradesman/craftsman", 19:  "unemployed", 20:  "writer"}

In [2]:
ratings = pd.read_csv('./datasets/movie_lens-1M/ratings.dat', 
                 delimiter='::', 
                 encoding='latin1', 
                 engine = 'python', 
                 header = None,
                 names = ['user_id', 'movie_id', 'rating', 'timestamp']
)
movies = pd.read_csv('./datasets/movie_lens-1M/movies.dat', 
                 delimiter='::', 
                 encoding = 'latin1',
                 header = None,
                 engine = 'python',
                 names = ['movie_id', 'title', 'genres']
)
users = pd.read_csv('./datasets/movie_lens-1M/users.dat', 
                 delimiter='::', 
                 encoding='latin1', 
                 engine = 'python', 
                 header = None,
                 names = ['user_id', 'gender', 'age', 'occupation', 'zip_code']
)

In [3]:
from datetime import datetime

# TODO: later change to hive.sql
ratings['timestamp'] = ratings['timestamp'].apply(lambda timestamp: datetime.fromtimestamp(timestamp))

ratings = ratings.sort_values(['rating', 'timestamp'], ascending = [False, False])
ratings.head()

,user_id,movie_id,rating,timestamp
825438,4958,1407,5,2003-02-28 09:47:23
984025,5948,1180,5,2003-02-28 05:11:19
984632,5950,3196,5,2003-02-27 09:54:22
984849,5950,111,5,2003-02-27 09:50:41
984825,5950,2329,5,2003-02-27 09:45:48


In [4]:
movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
users.head()

,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


## Create merged ratings and analysis

In [14]:
# merge datasets
merged = pd.merge(ratings, users, how = 'inner', on = 'user_id')
merged = pd.merge(merged, movies, how = 'inner', on = 'movie_id')
merged.head()

merged = merged[['user_id', 'movie_id', 'rating', 'title']].sort_values(by=['user_id', 'movie_id', 'rating'])
print(merged.head())

print(f"Unique number of users in the dataset: {merged['user_id'].nunique()}")
print(f"Unique number of movies in the dataset: {merged['movie_id'].nunique()}")

        user_id  movie_id  rating                                      title
171933        1         1       5                           Toy Story (1995)
911414        1        48       5                          Pocahontas (1995)
21420         1       150       5                           Apollo 13 (1995)
49104         1       260       4  Star Wars: Episode IV - A New Hope (1977)
34221         1       527       5                    Schindler's List (1993)
Unique number of users in the dataset: 6040
Unique number of movies in the dataset: 3706


#### Some movies in the movie dataset does not have any ratings.
These movies will not be used as input data.

In [7]:
# hence there are some movies missing in merged dataset
ratings[ratings['movie_id'] == 51]

,user_id,movie_id,rating,timestamp


In [8]:
missing_movies = movies[~movies['movie_id'].isin(merged['movie_id'].unique())]
missing_movies

,movie_id,title,genres
50,51,Guardian Angel (1994),Action|Drama|Thriller
107,109,Headless Body in Topless Bar (1995),Comedy
113,115,Happiness Is in the Field (1995),Comedy
141,143,Gospa (1995),Drama
281,284,New York Cop (1996),Action|Crime
...,...,...,...
3581,3650,Anguish (Angustia) (1986),Horror
3681,3750,Boricua's Bond (2000),Drama
3759,3829,Mad About Mambo (2000),Comedy|Romance
3786,3856,Autumn Heart (1999),Drama


## Train Test Split

### Train test split

In [15]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train, test = train_test_split(merged, test_size=0.15, random_state=42)

print(f"Unique number of users in the train dataset: {train['user_id'].nunique()}")
print(f"Unique number of movies in the train dataset: {train['movie_id'].nunique()}")

print(f"Unique number of users in the train dataset: {test['user_id'].nunique()}")
print(f"Unique number of movies in the train dataset: {test['movie_id'].nunique()}")

train

Unique number of users in the train dataset: 6040
Unique number of movies in the train dataset: 3705
Unique number of users in the train dataset: 3749
Unique number of movies in the train dataset: 2274


,user_id,movie_id,rating,title
602236,4779,1912,4,Out of Sight (1998)
743411,36,2273,4,Rush Hour (1998)
791191,5961,2600,4,eXistenZ (1999)
701462,195,1882,4,Godzilla (1998)
126990,3283,2396,4,Shakespeare in Love (1998)
...,...,...,...,...
381337,1586,3507,5,"Odd Couple, The (1968)"
927941,2129,2790,3,"Final Conflict, The (a.k.a. Omen III: The Fina..."
302949,854,1704,4,Good Will Hunting (1997)
91883,4033,1199,5,Brazil (1985)


### Export datasets

In [16]:
merged.to_csv('./datasets/merged.csv', index = False)
train.to_csv('./datasets/train.csv', index=False)
test.to_csv('./datasets/test.csv', index=False)